From : https://docs.python.org/3.6/reference/index.html

In [ ]:
import os
os.getpid()

In [ ]:
import inspect
import hybridcuda
cures = hybridcuda.initcuda()
hybridcuda.registerheader("hybpython.cuh", os.getcwd() + os.sep + ".." + os.sep + ".." + os.sep + "hybpython.cuh")
assert cures == 0

In [ ]:
class hybridkernel:
    gridDimX = 1
    blockDimX = 1
    shared = 0
    stream = 0
    def __init__(self, func):
        self.hc = hybridcuda.processfunction(func)
        self.hc = hybridcuda.cudajitcode(self.hc)
        self.hc = hybridcuda.ptxlinkcode(self.hc)
        
    def __call__(self, *args):
        self.hc = hybridcuda.launch(self.hc, self.gridDimX,1,1, self.blockDimX,1,1, self.shared,self.stream, *args)

    def __getitem__(self, args):
        if (type(args) != tuple):
            self.grid = args
            return self
        # args is a tuple...
        if (len(args) > 0):
            self.grid = args[0]
        if (len(args) > 1):
            self.block = args[1]
        if (len(args) > 2):
            self.shared = args[2]
        if (len(args) > 3):
            self.stream = args[3]
        return self

#decorator definition
def hybridfunction(func):
    return hybridkernel(func)

# 6. Expressions
https://docs.python.org/3/reference/expressions.html


## 6.13 Lambdas

https://docs.python.org/3.6/reference/expressions.html#lambda

In [ ]:
a = lambda : 42
hybridcuda.processlambda(a)
getattr(a,'__hybrid_cuda__')

In [ ]:
print(getattr(a,'__hybrid_cuda__')['cuda'])

In [ ]:
a = lambda x,y: x+y
print (a(12,30))

In [ ]:
hybridcuda.processlambda(a)
print(getattr(a,'__hybrid_cuda__')['cuda'])

In [ ]:
def lambda_use(func,a,b,c):
    c[0] = func(a,b)

In [ ]:
hc = hybridcuda.processfunction(lambda_use)
print(hc['cuda'])

In [ ]:
@hybridfunction
def lambda_expression(func,a,b,c):
    c[0] = func(a,b)

In [ ]:
print(lambda_expression.hc['cuda'])

In [ ]:
d = [0]

z = lambda x,y:x+y

lambda_expression[1,1](z,12,30,d)
assert d[0] == 42

In [ ]:
d

In [ ]:
dir(z)

### Lambdas with capture

In [ ]:
def func(x,y):
    return lambda a,b: y[0]+x

d = [30]
z = func(12,d)
print(z(1,2))

d[0] = 60
print(z(1,2))


In [ ]:
@hybridfunction
def captured_lambda_expression(func,a,b,c):
    c[0] = func(a,b)

In [ ]:
print(captured_lambda_expression.hc['cuda'])

In [ ]:
c = [0]
captured_lambda_expression[1,1](z,1,2,c)
print(c[0])

In [ ]:
print(getattr(z,'__hybrid_cuda__')['cuda'])

In [ ]:
import dis
dis.dis(getattr(z,'__code__'))

In [ ]:
for attr in dir(z):
    print(attr + ' = ' + str(getattr(z,attr)))

In [ ]:
import ast
import inspect
print(getattr(ast.parse, '__doc__'))

def func(x,y):
    return lambda : y[0]+x

d = [30]
z = func(12,d)

inspect.getsource(z)

# labmda has no argument
assert ast.parse(inspect.getsource(z).strip()).body[0].value.args.args == []
# when parsing ast, we get the used name => this information gets lost at runtime from the '__closure__'
ast.parse(inspect.getsource(z).strip()).body[0].value.body.left.value.id

# NEED TO COMPILED PYTHON !!

<hr/>

# SAND BOX BEYOIND THIS POINT...

### Inspecting what is a lambda precisely

#### Lambda capture

Lambda capture is not dynamic

In [ ]:
l = lambda x: w

In [ ]:
l(12)

In [ ]:
def f(lam):
    w = 42
    return lam(0)

In [ ]:
f(l)

But static capture is possible

In [ ]:
def func(x,y):
    z = lambda z : x+z
    return 42 + z(y)

In [ ]:
func(1,2)

In [ ]:
y

In [ ]:
x

Lambda can capture function parameters... (Support for this may not be easy...)

In [ ]:
def func2(x,y):
    return lambda z : x+y+z

In [ ]:
func2(1,2)(3)

In [ ]:
import dis
dis.dis(func2(1,2))

In [ ]:
import inspect
lines = inspect.getsourcelines(func2(1,2))
print("".join(lines[0]))

In [ ]:
dir(func2(1,2))

In [ ]:
type(getattr(func2(1,2), '__closure__')[0])

In [ ]:
getattr(func2, '__closure__')

In [ ]:
arg1 = 1
arg2 = 2
lamb = func2(arg1,arg2)
lamb(42)

In [ ]:
arg2 = 3
lamb(42)

In [ ]:
def func3(x):
    return lambda z : x[0]+z

In [ ]:
arg3 = [12]
lamb = func3(arg3)
lamb(12)

In [ ]:
arg3[0] = 15
lamb(12)